In [17]:
import torch
from torch.utils.data import DataLoader
from transformers import RagTokenForGeneration, RagTokenizer

# Initialize model and tokenizer
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Simulate a dataset
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, input_texts, labels):
        self.input_texts = input_texts
        self.labels = labels

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        label = self.labels[idx]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.squeeze(0)
        label_ids = tokenizer(label, return_tensors="pt").input_ids.squeeze(0)
        return {
            "input_ids": input_ids,
            "attention_mask": (input_ids != tokenizer.pad_token_id).long(),
            "labels": label_ids
        }

# Example input and labels
input_texts = ["How to train a model?", "What is transformers library?"]
labels = ["You can use various algorithms.", "It is a Python library."]

# Create dataset and dataloader
dataset = SimpleDataset(input_texts, labels)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)  # Ensure batch size is a multiple of 5

# Set n_docs to match the batch size
n_docs = 5

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Create doc_scores (simulated as zeros, since we're not using a retriever)
        doc_scores = torch.zeros(input_ids.size(0), 1).to(device)  # 1 document per batch

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
            doc_scores=doc_scores,
            n_docs=n_docs,  # Pass n_docs here
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


pytorch_model.bin:  14%|#3        | 283M/2.06G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called fr

AttributeError: 'RagTokenizer' object has no attribute 'pad_token_id'

In [21]:
# Cell 1: Setup and Imports
import torch
from transformers import RagTokenForGeneration, RagTokenizer, RagRetriever, RagConfig
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm.notebook import tqdm
import logging
import os

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configuration
CONFIG = {
    "model_name": "facebook/rag-sequence-nq",
    "max_length": 512,
    "batch_size": 4,
    "num_epochs": 3,
    "learning_rate": 1e-5
}

# Cell 2: Dataset Class and Loading
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []
        
        # Pre-tokenize the dataset
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparing dataset"):
            try:
                article = row["articles"]
                question = "What is the content of this article?"
                
                # Tokenize input
                inputs = self.tokenizer.question_encoder(
                    question,
                    max_length=self.max_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt",
                )
                
                # Tokenize target
                targets = self.tokenizer(
                    article,
                    max_length=self.max_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt",
                )
                
                self.inputs.append({
                    "input_ids": inputs["input_ids"].squeeze(),
                    "attention_mask": inputs["attention_mask"].squeeze(),
                })
                self.targets.append(targets["input_ids"].squeeze())
                
            except Exception as e:
                logging.error(f"Error processing row: {e}")
                continue

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load model components
tokenizer = RagTokenizer.from_pretrained(CONFIG["model_name"])

# Configure RAG
config = RagConfig.from_pretrained(CONFIG["model_name"])
config.index_name = "custom"  # Changed from "exact"
config.use_dummy_dataset = True

# Initialize retriever with configuration and trust_remote_code
retriever = RagRetriever.from_pretrained(
    CONFIG["model_name"],
    index_name="custom",
    use_dummy_dataset=True,
    config=config,
    trust_remote_code=True  # Added this parameter
)

# Initialize model
model = RagTokenForGeneration.from_pretrained(CONFIG["model_name"], config=config)
model.set_retriever(retriever)

# Load dataset
articles = pd.read_csv("articles.csv", usecols=["articles"])
dataset = CQADataset(articles, tokenizer, max_length=CONFIG["max_length"])
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True)

# Cell 3: Training Loop
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])

# Training loop
for epoch in range(CONFIG["num_epochs"]):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{CONFIG['num_epochs']}")
    
    for batch in progress_bar:
        # Zero gradients
        optimizer.zero_grad()
        
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")

# Save the model
os.makedirs("models", exist_ok=True)
model.save_pretrained("models/rag_model_final")
tokenizer.save_pretrained("models/rag_model_final")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

ValueError: Please provide `dataset_path` and `index_path` after calling `dataset.save_to_disk(dataset_path)` and `dataset.get_index('embeddings').save(index_path)`.

In [20]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.

In [ ]:
import torch
from transformers import RagTokenForGeneration, RagTokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Load RAG model and tokenizer
model_name = "facebook/rag-sequence-nq"
model = RagTokenForGeneration.from_pretrained(model_name)
tokenizer = RagTokenizer.from_pretrained(model_name)

# Dataset class
class CQADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inputs = []
        self.targets = []

        # Pre-tokenize the entire dataset to avoid repeated tokenization in each epoch
        for _, row in df.iterrows():
            article = row["articles"]

            # Create a dummy question for each article (you can customize this)
            question = "What is the content of this article?"

            # Dummy answer can be the article itself or a summary, depending on your use case
            answer = article

            input_text = f"Context: {article} Reasoning: {question}"
            target_text = answer

            # Tokenize input and target texts and store the results
            inputs = self.tokenizer(
                input_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            targets = self.tokenizer(
                target_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            self.inputs.append({
                "input_ids": inputs["input_ids"].squeeze(),
                "attention_mask": inputs["attention_mask"].squeeze(),
            })
            self.targets.append(targets["input_ids"].squeeze())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"],
            "attention_mask": self.inputs[idx]["attention_mask"],
            "labels": self.targets[idx],
        }

# Load dataset (articles only)
articles = pd.read_csv("articles.csv", usecols=["articles"])

# Initialize dataset and dataloader
dataset = CQADataset(articles, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Create empty context tensors to disable the retrieval mechanism
        context_input_ids = torch.zeros_like(input_ids)  # Empty context input
        context_attention_mask = torch.zeros_like(attention_mask)  # Empty context mask

        # Perform forward pass with empty context input and mask
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AssertionError: Make sure that `doc_scores` are passed, if no `retriever` is set. Alternatively, you can set a retriever using the `set_retriever(...)` function.